In [ ]:
import os
import pickle
import numpy as np
import json
from myEstimator import gd_estimator, mymcmc_estimator2, ML_estimator
from scipy.stats import beta

folder = '/Users/anjiefang/Desktop/CIKM2017_select_best/'
INTERVALS = [30,60,120]
trainning_size = [0.5, 0.6, 0.8]
estimator_name = ['GDE', 'MLE', 'MCE']

fold_num = 5
partition_num = 1000
est_method = 'BFGS'

errors = [[[[] for e in estimator_name] for t in trainning_size ] for i in INTERVALS]

def rmse(data, dist, para):
    y_true, bins = np.histogram(data, bins=20, density=True)
    bins[0] = 0.0
    bins[-1] = 1.0
    est_a = np.array([(dist.cdf(bins[i + 1], para[0], para[1]) - dist.cdf(bins[i], para[0], para[1])) for i in range(len(bins) - 1)])
    step = np.array([(bins[i + 1] - bins[i]) for i in range(len(bins) - 1)])
    if np.sum(est_a) < 1e-5:
        return 2.0
    tru_a = y_true * step
    return np.sqrt(np.mean((est_a - tru_a)**2))

for i in range(len(INTERVALS)):
    print 'Time Interval: ' + str(INTERVALS[i])
    periods = np.array(pickle.load(open(folder + 'time_period_' + str(INTERVALS[i]) + '/periods.p')))
    for p in range(len(periods)):
        for t in range(len(trainning_size)):
            data_full = periods[p]
            data = np.array([tmp for tmp in data_full if tmp <= trainning_size[t]])
            
            gde = gd_estimator(data = data)
            gd_res = gde.estimate(fold_num=fold_num, partition_num=partition_num)
            err0 = rmse(data_full, beta, gd_res)
            
            mle = ML_estimator(data = data)
            ml_res = mle.estimate(fold_num = fold_num, partition_num = partition_num)
            err1 = rmse(data_full, beta, ml_res)
            
            mce = mymcmc_estimator2(data = data)
            mc_res = mce.estimate(fold_num * 5)
            err2 = rmse(data_full, beta, mc_res)
            
            errors[i][t] = [err0, err1, err2]
        print errors[i]

In [13]:
import os
import pickle
import numpy as np
import json
from myEstimator import gd_estimator, mymcmc_estimator2, ML_estimator
from scipy.stats import beta

res_col = 0
output_folder = '/Volumes/anjie/tr.anjie/CIKM2017/output'

INTERVALS = [30,60,120]
trainning_size = ['0.3', '0.5', '0.8']
estimator_name = ['BSE' ,'RNE' ,'GDE', 'MLE', 'MCE']
errors = {m:{i:{s:[] for s in trainning_size} for i in INTERVALS} for m in estimator_name}
for f in os.listdir(output_folder):
    if 'txt' not in f: continue
    method, interval, size, index = f.split('.txt')[0].split('_')
    index = int(index)
    interval = int(interval)
    res = float(open(output_folder + '/' + f).readline().split(',')[res_col])
    if np.isnan(res): continue
    errors[method][interval][size].append(res)
    
mini_sample = min([len(errors[m][i][s]) for m in estimator_name for i in INTERVALS for s in trainning_size])
maxi_sample = max([len(errors[m][i][s]) for m in estimator_name for i in INTERVALS for s in trainning_size])
print mini_sample, maxi_sample
   

194 200


In [14]:
for size in trainning_size:
    print '**************'
    for m in estimator_name:
        for interval in INTERVALS:
            data = np.array(errors[m][interval][size])
            print 'Size: ' + size + ', Method: ' + m + ', Interval: ' + str(interval) + ', Error: ' + str(np.mean(data))

**************
Size: 0.3, Method: BSE, Interval: 30, Error: 0.336307565833
Size: 0.3, Method: BSE, Interval: 60, Error: 0.312735584127
Size: 0.3, Method: BSE, Interval: 120, Error: 0.332067977202
Size: 0.3, Method: RNE, Interval: 30, Error: 0.336557033998
Size: 0.3, Method: RNE, Interval: 60, Error: 0.31279913991
Size: 0.3, Method: RNE, Interval: 120, Error: 0.32796082776
Size: 0.3, Method: GDE, Interval: 30, Error: 1.01791843763
Size: 0.3, Method: GDE, Interval: 60, Error: 0.973264271059
Size: 0.3, Method: GDE, Interval: 120, Error: 0.955396220903
Size: 0.3, Method: MLE, Interval: 30, Error: 1.04083744785
Size: 0.3, Method: MLE, Interval: 60, Error: 0.994069744178
Size: 0.3, Method: MLE, Interval: 120, Error: 0.978299056298
Size: 0.3, Method: MCE, Interval: 30, Error: 0.495084162236
Size: 0.3, Method: MCE, Interval: 60, Error: 0.507342471681
Size: 0.3, Method: MCE, Interval: 120, Error: 0.531053627138
**************
Size: 0.5, Method: BSE, Interval: 30, Error: 0.23749308528
Size: 0.5,